In [4]:
import pandas as pd

In [5]:
user_table = pd.read_csv ("user_table.csv")
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [6]:
home_page = pd.read_csv("home_page_table.csv")
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [7]:
search_page = pd.read_csv("search_page_table.csv")
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [8]:
payment_page = pd.read_csv("payment_page_table.csv")
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [9]:
payment_confirmation = pd.read_csv("payment_confirmation_table.csv")
payment_confirmation.head()


,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [10]:
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [23]:
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [24]:
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [25]:
df["sex"].value_counts()

Male      45325
Female    45075
Name: sex, dtype: int64

In [26]:
df.describe()

,user_id,home_page,search_page,payment_page,payment_confirmation
count,90400.000000,90400.0,90400.000000,90400.000000,90400.000000
mean,498709.683473,1.0,0.500000,0.066704,0.005000
std,289040.674085,0.0,0.500003,0.249509,0.070534
min,17.000000,1.0,0.000000,0.000000,0.000000
25%,247978.750000,1.0,0.000000,0.000000,0.000000
50%,498267.000000,1.0,0.500000,0.000000,0.000000
75%,749789.250000,1.0,1.000000,0.000000,0.000000
max,999979.000000,1.0,1.000000,1.000000,1.000000


In [27]:
from plotly import graph_objects as go

In [28]:
dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()
dfg.head()

sex,index,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [29]:
dfg.columns.name = None
dfg.head()

,index,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [30]:
dfg = dfg.rename(columns = {'index':'Page'})
dfg.head()

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [21]:
fig = go.Figure()
fig.add_trace(go.Funnel(y = dfg['Page'], x = dfg['Female'],name = 'Female', constraintext='outside', textinfo = 'value+percent previous'))

fig.add_trace(go.Funnel(y = dfg['Page'],x = dfg['Male'],name = 'Male', constraintext='outside', textinfo = 'value+percent previous')) 
fig.update_layout(title = 'Воронка конверсии по полу', title_x=0.5, autosize=False, width=1000,height=600)

fig.show()

In [33]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 90400 entries, 0 to 90399
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   user_id               90400 non-null  int64 
 1   date                  90400 non-null  object
 2   device                90400 non-null  object
 3   sex                   90400 non-null  object
 4   home_page             90400 non-null  int64 
 5   search_page           90400 non-null  int64 
 6   payment_page          90400 non-null  int64 
 7   payment_confirmation  90400 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 6.2+ MB


In [35]:
df['date'] = df['date'].astype('datetime64[ns]')

In [38]:
df['month'] = df['date'].dt.month
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1


In [39]:
df['day_of_week'] = df['date'].dt.dayofweek
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day_of_week
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2,5
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1,1
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4,3
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2,2
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1,3


In [47]:
dfg = df.groupby('month')[['payment_confirmation']].sum().reset_index()
dfg.head(1)

,month,payment_confirmation
0,1,189


In [48]:
dfg = df.groupby('day_of_week')[['payment_confirmation']].sum().reset_index()
dfg.head(1)

,day_of_week,payment_confirmation
0,0,77


In [53]:
df.pivot_table(index = ['sex','day_of_week'], values = 'payment_confirmation')

payment_confirmation
sex    day_of_week                      
Female 0                        0.006719
       1                        0.006240
       2                        0.004718
       3                        0.004608
       4                        0.004109
       5                        0.005675
       6                        0.005380
Male   0                        0.005187
       1                        0.004613
       2                        0.003708
       3                        0.005211
       4                        0.004360
       5                        0.005266
       6                        0.004208

In [56]:
df.groupby(['sex', 'day_of_week'])['payment_confirmation'].sum().unstack('sex')

sex,Female,Male
day_of_week,,
0,43,34
1,40,29
2,30,24
3,31,35
4,26,28
5,36,34
6,35,27
